In [1]:
from llm_engines import ChatgptLLM, LLMApi
from agent_factory import create_groupchat, unique_topics, get_persona_by_topics, gen_name
import random, time, textwrap, os
from IPython.display import HTML, display


# first of all, create a thesis_generations folder
if not os.path.exists('thesis_generations'):
    os.makedirs('thesis_generations')


# extract 100 unique topics or load them from a file if it already exists
if not os.path.exists('thesis_generations/sampled_topics.txt'):
    # get Southern Ice Tea out of the way
    unique_topics.remove('Southern Ice Tea')
    sampled_topics = random.sample(unique_topics, 100)

    with open('thesis_generations/sampled_topics.txt', 'w') as f:
        for topic in sampled_topics:
            f.write(topic + '\n')
else:
    with open('thesis_generations/sampled_topics.txt', 'r') as f:
        sampled_topics = f.read().splitlines()


## Naive conversation generation

In [2]:
# pick 100 persona couples based on the sampled topics but only if they don't already exist
if not os.path.exists('thesis_generations/personas_pairs.json'):
    personas_pairs = []
    for topic in sampled_topics:
        persona1 = get_persona_by_topics([topic])
        persona2 = get_persona_by_topics([topic])
        # crete a name for the couple
        name1 = gen_name(persona=persona1)
        name2 = gen_name(persona=persona2)
        personas_pairs.append({'name1': name1, 'persona1': persona1, 'name2': name2, 'persona2': persona2, 'topic': topic})
    

In [3]:

# load the persona pairs from the file
from agent_factory import gen_name
import json

if os.path.exists('thesis_generations/personas_pairs.json'):
    with open('thesis_generations/personas_pairs.json', 'r') as f:
        personas_pairs = json.load(f)
else:
    personas_pairs = [] 

    for topic in sampled_topics:
        persona1 = get_persona_by_topics([topic])
        persona2 = get_persona_by_topics([topic])
        # crete a name for the couple
        name1 = gen_name(persona=persona1)
        name2 = gen_name(persona=persona2)
        personas_pairs.append({'name1': name1, 'persona1': persona1, 'name2': name2, 'persona2': persona2, 'topic': topic})
    with open('thesis_generations/personas_pairs.json', 'w') as f:
        json.dump(personas_pairs, f)
    
    
from places_replication import NaiveConversationAgent

# for each pair, create a group chat and run the PlacesReplication
groupchats = []


for pair in personas_pairs:
    groupchat = [NaiveConversationAgent(name=pair['name1'], persona=pair['persona1']), NaiveConversationAgent(name=pair['name2'], persona=pair['persona2'])]
    groupchats.append(groupchat)

# let's check how many groupchats we have
len(groupchats)

100

In [6]:
# check if two agents with the same persona are in the same groupchat

for i, groupchat in enumerate(groupchats):
    tries=0
    while groupchat[0].persona == groupchat[1].persona:
        groupchat[1].persona = get_persona_by_topics([personas_pairs[i]['topic']])
        groupchats[i] = groupchat
        print('changed persona of agent 2 in groupchat', i)
        tries += 1
        if tries > 10:
            print("critical topic:", personas_pairs[i]['topic'])
            break   

# let's check if the personas are different
for i, groupchat in enumerate(groupchats):
    assert groupchat[0].persona != groupchat[1].persona, 'personas are the same in groupchat ' + str(i)

# check that names are not longer than 20 characters
for groupchat in groupchats:
    for agent in groupchat:
        if len(agent.name) > 20:
            print('name too long:', agent.name)
            agent.name = gen_name(agent.persona)
            
# check that the agents have different names
for i, groupchat in enumerate(groupchats):
    while groupchat[0].name == groupchat[1].name:
        groupchat[1].name = gen_name(groupchat[1].persona)
        groupchats[i] = groupchat
        print('changed name of agent 2 in groupchat', i)
    

# in each persona, substitute $name$ with the name of the agent
for groupchat in groupchats:
    for agent in groupchat:
        agent.persona = agent.persona.replace('$name$', agent.name)

from Levenshtein import distance

# check that the agents have different names
for i, groupchat in enumerate(groupchats):
    # while names are one edit away from each other
    while distance(groupchat[0].name, groupchat[1].name) < 2:    
        groupchat[1].name = gen_name(groupchat[1].persona)
        groupchats[i] = groupchat
        print('changed name of agent 2 in groupchat', i)
# let's save the personas_pairs with the updated personas

with open('thesis_generations/personas_pairs.json', 'w') as f:
    personas_pairs = [{'name1': groupchat[0].name, 'persona1': groupchat[0].persona, 'name2': groupchat[1].name, 'persona2': groupchat[1].persona, 'topic': personas_pairs[i]['topic']} for i, groupchat in enumerate(groupchats)]
    json.dump(personas_pairs, f)

changed name of agent 2 in groupchat 88


In [7]:
# for each groupchat, run the generation using the local model
from places_replication import NaiveConversationGeneration
from tqdm.notebook import tqdm

chat_ids_list = []

# check if the chat_ids_list file already exists
if os.path.exists('thesis_generations/naive_gen_chat_ids_list.json'):
    with open('thesis_generations/naive_gen_chat_ids_list.json', 'r') as f:
        chat_ids_list = json.load(f)

# get the index to resume from
resume_index = len(chat_ids_list)

# iterate over the remaining groupchats
for i, groupchat in enumerate(tqdm(groupchats[resume_index:])):
    try:
        convo = NaiveConversationGeneration(agent_list=groupchat, neutral_llm=LLMApi())
        # cache the chat_id for easier retrieval
        chat_ids_list.append(convo.chat_id)
        convo.generate_conversation(min_turns=10, start_conversation=True)
        # save the chat_ids_list
        with open('thesis_generations/naive_gen_chat_ids_list.json', 'w') as f:
            json.dump(chat_ids_list, f)

    except Exception as e:
        print(f"Error occurred during conversation generation for groupchat {i + resume_index}: {str(e)}")
        break


  0%|          | 0/12 [00:00<?, ?it/s]

Estella: How are you?
Orion:  I'm doing well, thank you! I brought a special bottle of wine
    to accompany your cosmic dish. It's a cosmic-themed wine, perfect for the
    occasion!
Estella:  That's so thoughtful of you, Orion! I can't wait to try it.
    I've been working on this recipe for quite some time now, and I'm excited to
    see how it pairs with the wine.
Orion:  I'm sure it will be an amazing experience, just like exploring
    the mysteries of the universe. Speaking of which, what inspired you to
    create a dish inspired by black holes?
Estella:  Well, I've always been fascinated by astrophysics and the
    enigmatic allure of black holes. I wanted to create a dish that captures the
    essence of their beauty and power, while also being mindful of the fact that
    they can be quite destructive.
Orion:  That's a fascinating concept. I'm excited to see how it
    translates into the culinary world.
Estella:  I used a variety of ingredients that represent the different


In [10]:
# load all the conversations
conversations = []

for chat_id in tqdm(chat_ids_list):
    path="chat_history/naive_chat_history_"+chat_id+".json"
    with open(path, 'r') as f:
        convo = json.load(f)
        conversations.append(convo)

  0%|          | 0/100 [00:00<?, ?it/s]

In [11]:
conversations

[[[0, 'Vivian', 'Hi!'],
  [1, 'Jasper', ' Hey, how are you doing today?'],
  [2,
   'Vivian',
   " I'm doing well, just got back from a morning jog and feeling energized! How about you?"],
  [3,
   'Jasper',
   " I'm good too, thanks for asking! I've been fascinated by studying tomato plants' growth patterns recently."],
  [4,
   'Vivian',
   " Tomato plants, huh? That's an interesting subject! I've been reading up on Marxist theories and how they tie into urban art, specifically city murals."],
  [5,
   'Jasper',
   " That sounds fascinating as well! I've always admired vibrant city murals and the messages they often convey."],
  [6,
   'Vivian',
   ' Definitely! I went for a jog in the city today, and it felt like the murals were alive in their own way. It got me thinking about how art and politics can intertwine in urban landscapes.'],
  [7,
   'Jasper',
   " I can imagine. It's amazing how our separate interests, yours and mine, can somehow find connections with each other."],
  [8